# VotingClassifier (lightgbm, catboost, tensorflow) 

## 1. read datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import tensorflow as tf
import random
import os

os.environ['TF_DETERMINISTIC_OPS'] = '1'

SEED=42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
    
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')


## 2. preprocessing datasets.

### 2.1. convert 'target' value to 0,1,2,3. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label_map = {
    'Class_1' : 0,
    'Class_2' : 1,
    'Class_3' : 2,
    'Class_4' : 3,
}
train['target'] = train['target'].map(label_map)

### 2.2. clip datasets.

In [ ]:
features = ['feature_{}'.format(x) for x in range(50)]
qt = train[features].quantile(np.arange(0,1,0.002))

def clip(df):
    df = df.copy()
    for feature in features:
        df[feature] = df[feature].clip(lower=0, upper=qt.loc[0.998][feature])
    return df


### 2.3. prepare feature's value based on the probability of target's rate. 

In [ ]:
values=[]
labels=[0,1,2,3,]
for feature in features:
    grouped = clip(train).groupby(feature)
    for value, group in grouped:
        value=[feature, value]
        for label in labels:
            p =  (group['target'] == label).mean()
            p = np.clip(p, 1e-06, 1 - 1e-06)
            value.append(np.log(p+0.5))
            value.append(np.log(p/(1-p)))
        values.append(value)
df_proba = pd.DataFrame(values,
                        columns=['feature', 'value',
                                 'Class_1_proba1',
                                 'Class_1_proba2',
                                 'Class_2_proba1',
                                 'Class_2_proba2',
                                 'Class_3_proba1',
                                 'Class_3_proba2',
                                 'Class_4_proba1',
                                 'Class_4_proba2',
                                ])
proba_dict_1={}
proba_dict_2={}

for i in range(len(df_proba)):
    feature = df_proba.iloc[i]['feature']
    value = df_proba.iloc[i]['value']
    proba_dict_1[feature, value] = df_proba.iloc[i][['Class_1_proba1','Class_2_proba1','Class_3_proba1','Class_4_proba1',]].values.astype(float)
    proba_dict_2[feature, value] = df_proba.iloc[i][['Class_1_proba2','Class_2_proba2','Class_3_proba2','Class_4_proba2',]].values.astype(float)
    

## 3. define my transformer

In [ ]:
from sklearn.base import TransformerMixin

def reshape(df):
    values=[]
    for value in df.values:
        values.append([_ for _ in value])
    return np.array(values)

class MyTransformer1(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=qt.loc[0.002][feature], upper=qt.loc[0.998][feature])
            newX[feature] = 1 / (newX[feature] - newX[feature].min() + 1)
        return newX

class MyTransformer2(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        X = clip(X)
        for feature in features:
            newX[feature] = X[feature].apply(lambda x:proba_dict_1[feature, x])
        return reshape(newX).reshape((-1,200))

class MyTransformer3(MyTransformer2):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        X = clip(X)
        for feature in features:
            newX[feature] = X[feature].apply(lambda x:proba_dict_1[feature, x])
        return reshape(newX)

def normalize(df, columns):
    for column in columns:
        min_val, max_val = df[column].agg([min,max])
        df[column] = (df[column] - min_val) / (max_val - min_val)
    return df

class MyTransformer4(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=0).apply(lambda x:1/(x+1))
        return normalize(newX, features)


## 4. define my classifier

In [ ]:
from sklearn.base import ClassifierMixin
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline

my_model1 = CatBoostClassifier(
    iterations=887,
    min_child_samples=200,
    random_state=SEED,
    max_depth=6,
    verbose=0)

my_model2 = CatBoostClassifier(
    iterations=160,
    min_child_samples=200,
    max_depth=2,
    eval_metric='MultiClass',
    random_state=SEED,
    verbose=0)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule, 
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)
class TensorflowClassifier(ClassifierMixin):
    def __init__(self):
        self.histories=[]
        self.classes_ = [0,1,2,3]
        self.model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(50,4)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(4, activation='softmax')
        ])
        self.model.compile(
            optimizer=optimizer, 
            loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_crossentropy',])
    def get_params(self,deep):
        return {}
    def fit(self, X, y):
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        history = self.model.fit(X, y, epochs=300, batch_size = 200, validation_split=0.1, callbacks=[callback],verbose=0)
        self.histories.append(history)
        return self
    def predict_proba(self, X):
        return self.model.predict(X).reshape((-1, 4))

my_model3 = TensorflowClassifier()

my_model4 = LGBMClassifier(
    random_state=SEED,
    min_child_samples=150,
    n_estimators=74,
)

pipeline1 = make_pipeline(MyTransformer1(),my_model1)
pipeline2 = make_pipeline(MyTransformer2(),my_model2)
pipeline3 = make_pipeline(MyTransformer3(),my_model3)
pipeline4 = make_pipeline(MyTransformer4(),my_model4)


## 5. train VotingClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss,accuracy_score

estimators = [
    ('mod1', pipeline1),
    ('mod2', pipeline2),
    ('mod3', pipeline3),
    ('mod4', pipeline4),
]

skf = StratifiedKFold(shuffle=True, random_state=SEED)

X = train[features]
y = train['target']
scores=[]
for train_index, valid_index in skf.split(X, y):
    X_train, X_val = X.iloc[train_index],X.iloc[valid_index]
    y_train, y_val = y.iloc[train_index],y.iloc[valid_index]
    mod = VotingClassifier(
        estimators=estimators,
        voting = 'soft',
    ).fit(X_train, y_train)
    y_pred = mod.predict_proba(X_val)
    print('log_loss={}, accuracy={}'.format(log_loss(y_val, y_pred),
                                            accuracy_score(y_val, np.argmax(y_pred,axis=1))))
    scores.append([mod, log_loss(y_val, y_pred),accuracy_score(y_val, np.argmax(y_pred,axis=1))])
scores = pd.DataFrame(scores, columns=['mod','log_loss','accuracy'])
print(scores.describe())

## 6. create my submission file

In [ ]:
y_pred_test = np.zeros((len(test), 4))

for mod in scores['mod']:
    y_pred_test += mod.predict_proba(test[features])

y_pred_test /= len(scores)

submission = test[['id']].copy()
submission['Class_1'] = y_pred_test[:,0]
submission['Class_2'] = y_pred_test[:,1]
submission['Class_3'] = y_pred_test[:,2]
submission['Class_4'] = y_pred_test[:,3]
submission.to_csv('submission.csv', index=False)